<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-08 15:42:52
-------------------
qualified stocks: 92
with latest results: 28
still star stocks: 18
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  3.53 L
Current:  1.32 C
-------------------
Today PnL: -1.21 L (-0.9%)
Current PnL: -22.60 L (-15.37%)
CY Booked + Current PnL: -14.33 L (-9.75%)
-------------------
Total profit:  1.94 L
Total loss:  -24.54 L
-------------------
Total Booked + Current PnL: 12.65 L (10.54%)
Total Booked PnL: 35.25 L (29.36%)
Curr Year Booked PnL: 8.27 L (6.25%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.13 C
Est FTT PnL: 80.85 L (61.13%)
Deployed:  1.20 C
Current:  1.32 C
CAGR/XIRR %: 6.28%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,28.77,H-SC,13.83,165091.0,38891.0,7281.0,7.25,30.82,4.41,36.59,156.0,5.34,1.28,47.19,XY24,NTT,HEALTHCARE
36,HINDALCO,651.95,756.01,-22.38,H-LC,5.63,107640.0,3328.0,13326.0,-1.33,3.19,12.38,15.96,10.0,0.25,0.84,20.63,X5K,ATH,METALS
67,SBIN,760.30,863.00,-11.51,M-LC,3.92,213140.0,11660.0,15559.0,-0.11,5.79,7.30,13.51,62.0,0.75,1.66,19.11,XY25,NTT,BANKS
71,SIS,477.00,477.00,2255.44,H-SC,7.09,63368.0,-17722.0,17724.0,0.34,-21.85,27.97,0.00,157.0,-1.00,0.49,28.42,OX40N,NTT,MISC
82,TTKPRESTIG,769.29,770.00,-9.13,H-SC,11.15,83028.0,-17749.0,17843.0,-0.04,-17.61,21.49,0.09,145.0,-0.99,0.64,9.08,OX40N,NTT,DURABLES


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,DABUR,505.2,735.0,-8.62,H-MC,4.51,197828.0,800.0,88825.0,-1.92,0.41,44.9,45.49,100.0,0.01,1.54,13.5,XY24,BTT,FMCG


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,CIPLA,1492.70,1795.0,-22.50,H-LC,1.93,203555.0,-945.0,42360.0,-0.35,-0.46,20.81,20.25,9.0,-0.02,1.58,8.23,X40N,BTT,PHARMA
5,ASIANPAINT,2961.56,4250.0,-15.19,H-LC,2.43,168341.0,-33045.0,120667.0,-1.23,-16.41,71.68,43.51,19.0,-0.27,1.31,17.18,X40,BTT,PAINTS
53,LTIM,5564.16,7230.2,-6.24,H-LC,3.54,180432.0,-19878.0,79859.0,-0.51,-9.92,44.26,29.94,24.0,-0.25,1.40,24.65,X200,ATH,IT
10,BAJAJHFL,122.26,152.0,-11.24,H-LC,3.66,184287.0,-16219.0,64998.0,-0.58,-8.09,35.27,24.33,30.0,-0.25,1.43,6.27,X40N,BTT,FINANCE
57,NESTLEIND,2268.60,2755.0,-59.53,H-LC,3.85,128290.0,-137136.0,194039.0,-1.86,-51.67,151.25,21.44,26.0,-0.71,1.00,3.09,XY25,NTT,FMCG


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.0,28.77,H-SC,13.83,165091.0,38891.0,7281.0,7.25,30.82,4.41,36.59,156.0,5.34,1.28,47.19,XY24,NTT,HEALTHCARE
83,UJJIVANSFB,52.77,53.0,37.69,H-SC,20.68,119070.0,-23409.0,24028.0,1.99,-16.43,20.18,0.44,163.0,-0.97,0.92,40.54,OX40N,NTT,BANKS
23,COFFEEDAY,59.14,80.0,-53.72,L-SC,21.91,69216.0,-44333.0,84381.0,1.72,-39.04,121.91,35.27,267.0,-0.53,0.54,69.41,XR,NTT,HOTELS
8,AWL,325.39,485.0,-69.74,H-MC,5.25,209674.0,-58773.0,190447.0,1.38,-21.89,90.83,49.05,82.0,-0.31,1.63,7.80,XY24,NTT,FMCG
86,VAIBHAVGBL,350.21,521.0,43.25,H-SC,7.87,95758.0,-56233.0,130355.0,1.13,-37.00,136.13,48.77,127.0,-0.43,0.74,10.69,XR,NTT,JEWELLERY


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,LAOPALA,369.40,464.00,112.30,H-SC,3.52,67540.0,-33306.0,59131.0,-6.38,-33.03,87.55,25.61,141.0,-0.56,0.52,25.22,AR,NTT,CERAMICS
80,TITAGARH,1097.23,1548.00,-14.37,H-SC,6.21,140516.0,-51499.0,130385.0,-5.97,-26.82,92.79,41.08,158.0,-0.39,1.09,18.93,XY24,NTT,ENGINEERING
27,DIXON,14367.00,18940.15,-31.08,H-MC,6.51,126912.0,11976.0,24608.0,-4.80,10.42,19.39,31.83,85.0,0.49,0.99,38.55,X40N,ATH,IT
43,INDIGOPNTS,1407.73,1408.00,118.19,H-SC,3.97,142898.0,-31661.0,31695.0,-4.77,-18.14,22.18,0.02,135.0,-1.00,1.11,23.91,OX40N,NTT,PAINTS
76,TANLA,917.30,1963.11,-34.11,H-SC,7.89,161402.0,-78013.0,350969.0,-3.67,-32.58,217.45,114.01,133.0,-0.22,1.25,48.03,AR,ATH,IT


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,UJJIVANSFB,52.77,53.0,37.69,H-SC,20.68,119070.0,-23409.0,24028.0,1.99,-16.43,20.18,0.44,163.0,-0.97,0.92,40.54,OX40N,NTT,BANKS
82,TTKPRESTIG,769.29,770.0,-9.13,H-SC,11.15,83028.0,-17749.0,17843.0,-0.04,-17.61,21.49,0.09,145.0,-0.99,0.64,9.08,OX40N,NTT,DURABLES
43,INDIGOPNTS,1407.73,1408.0,118.19,H-SC,3.97,142898.0,-31661.0,31695.0,-4.77,-18.14,22.18,0.02,135.0,-1.00,1.11,23.91,OX40N,NTT,PAINTS
77,TATAELXSI,7332.28,7332.0,-23.19,H-MC,2.16,80633.0,-22019.0,22013.0,-1.80,-21.45,27.30,-0.00,101.0,-1.00,0.63,22.57,OX40N,NTT,IT
71,SIS,477.00,477.0,2255.44,H-SC,7.09,63368.0,-17722.0,17724.0,0.34,-21.85,27.97,0.00,157.0,-1.00,0.49,28.42,OX40N,NTT,MISC


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TCS,3794.03,4998.00,-27.38,H-LC,15.38,236839.0,-59095.0,152998.0,-0.35,-19.97,64.60,31.73,1.0,-0.39,1.84,1.11,X40,BTT,IT
88,VBL,492.64,671.64,-12.12,H-LC,11.03,266821.0,1781.0,94508.0,-1.34,0.67,35.42,36.33,3.0,0.02,2.07,14.06,X40N,ATH,FMCG
45,INFY,1461.46,2275.00,-20.67,H-LC,9.51,254735.0,-6866.0,152484.0,-0.96,-2.62,59.86,55.67,4.0,-0.05,1.98,3.32,X40,BTT,IT
1,ABB,5551.76,7934.00,-45.12,H-LC,9.47,195858.0,-20661.0,113558.0,-1.70,-9.54,57.98,42.91,5.0,-0.18,1.52,2.57,AR,NTT,ELECTRICAL
34,HCLTECH,1562.67,1943.91,1.41,H-LC,9.90,180035.0,-10611.0,57125.0,0.01,-5.57,31.73,24.40,8.0,-0.19,1.40,9.48,X40,ATH,IT


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.0,28.77,H-SC,13.83,165091.0,38891.0,7281.0,7.25,30.82,4.41,36.59,156.0,5.34,1.28,47.19,XY24,NTT,HEALTHCARE


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,28.77,H-SC,13.83,165091.0,38891.0,7281.0,7.25,30.82,4.41,36.59,156.0,5.34,1.28,47.19,XY24,NTT,HEALTHCARE
50,KPIGREEN,497.21,731.64,24.70,H-SC,13.18,127625.0,2328.0,56742.0,-1.23,1.86,44.46,47.15,144.0,0.04,0.99,59.67,MH,ATH,POWER
35,HEROMOTOCO,4311.81,5949.07,0.00,H-MC,6.04,160993.0,10080.0,47219.0,-1.31,6.68,29.33,37.97,90.0,0.21,1.25,33.67,AR,ATH,AUTO
15,BLUESTARCO,1646.70,2326.38,-4.05,H-SC,7.95,180030.0,15360.0,52605.0,-1.49,9.33,29.22,41.28,119.0,0.29,1.40,18.05,X40N,ATH,AC
19,CAMS,3643.00,5211.76,-6.58,H-SC,2.41,104860.0,2856.0,41074.0,-1.65,2.80,39.17,43.06,120.0,0.07,0.81,20.40,X40N,ATH,MISC


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,0.00,H-MC,6.04,160993.0,10080.0,47219.0,-1.31,6.68,29.33,37.97,90.0,0.21,1.25,33.67,AR,ATH,AUTO
42,INDIAMART,2327.09,4850.92,-47.89,H-SC,13.34,135749.0,12413.0,121346.0,0.04,10.06,89.39,108.45,118.0,0.10,1.05,34.67,AR,ATH,MISC
50,KPIGREEN,497.21,731.64,24.70,H-SC,13.18,127625.0,2328.0,56742.0,-1.23,1.86,44.46,47.15,144.0,0.04,0.99,59.67,MH,ATH,POWER
90,WHIRLPOOL,1219.98,2270.00,-43.60,M-SC,4.90,94845.0,3346.0,75402.0,-2.32,3.66,79.50,86.07,211.0,0.04,0.74,37.20,XR,NTT,DURABLES
91,WIPRO,243.46,420.00,-13.30,M-LC,6.52,148211.0,-2734.0,112196.0,-1.35,-1.81,75.70,72.51,55.0,-0.02,1.15,4.10,XR,NTT,IT


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
71,SIS,477.00,477.00,2255.44,H-SC,7.09,63368.0,-17722.0,17724.0,0.34,-21.85,27.97,0.00,157.0,-1.00,0.49,28.42,OX40N,NTT,MISC
23,COFFEEDAY,59.14,80.00,-53.72,L-SC,21.91,69216.0,-44333.0,84381.0,1.72,-39.04,121.91,35.27,267.0,-0.53,0.54,69.41,XR,NTT,HOTELS
3,ALKYLAMINE,2347.98,4546.37,-4.67,H-SC,17.87,88756.0,-12207.0,106738.0,-2.10,-12.09,120.26,93.63,149.0,-0.11,0.69,32.20,SR,ATH,CHEMICALS
54,MASFIN,326.60,399.50,-20.05,H-SC,15.99,90720.0,-7260.0,29130.0,-0.18,-7.41,32.11,22.32,151.0,-0.25,0.70,31.58,XR,ATH,FINANCE
83,UJJIVANSFB,52.77,53.00,37.69,H-SC,20.68,119070.0,-23409.0,24028.0,1.99,-16.43,20.18,0.44,163.0,-0.97,0.92,40.54,OX40N,NTT,BANKS


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,28.77,H-SC,13.83,165091.0,38891.0,7281.0,7.25,30.82,4.41,36.59,156.0,5.34,1.28,47.19,XY24,NTT,HEALTHCARE
51,LALPATHLAB,2838.53,3545.00,-23.82,H-SC,8.04,216940.0,23920.0,24124.0,-0.71,12.39,11.12,24.89,128.0,0.99,1.69,38.63,X40N,NTT,HEALTHCARE
69,SHALBY,261.39,327.00,935.09,M-SC,8.17,140866.0,-40277.0,85745.0,-1.19,-22.24,60.87,25.10,233.0,-0.47,1.09,12.89,XY24,NTT,HEALTHCARE
35,HEROMOTOCO,4311.81,5949.07,0.00,H-MC,6.04,160993.0,10080.0,47219.0,-1.31,6.68,29.33,37.97,90.0,0.21,1.25,33.67,AR,ATH,AUTO
88,VBL,492.64,671.64,-12.12,H-LC,11.03,266821.0,1781.0,94508.0,-1.34,0.67,35.42,36.33,3.0,0.02,2.07,14.06,X40N,ATH,FMCG


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.49
1,25,41.36
2,50,71.40


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    40.97
LC    36.60
MC    22.41
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.26
X40N     15.36
X40      13.53
XY25     10.84
XR        9.03
OX40N     7.71
AR        7.70
X5K       2.26
MH        1.76
X200      1.40
SR        1.13
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    28.33
H-SC    27.59
H-MC    19.43
M-SC    11.93
M-LC     7.18
M-MC     2.63
L-SC     1.45
L-LC     1.09
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.39,-11.35,47.15
IT,12.07,-20.11,78.44
FINANCE,8.40,-20.52,64.24
BANKS,7.66,-15.69,62.90
PAINTS,5.78,-17.33,40.45
HEALTHCARE,5.27,3.27,21.15
MISC,5.18,-22.12,93.50
ELECTRICAL,4.76,-14.59,64.21
AUTO,4.44,-18.27,69.72


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2871395.0,23
XR,1130985.0,13
AR,1070868.0,9
X40,830641.0,10
XY25,654112.0,8
X40N,628838.0,12
OX40N,384275.0,10
SR,197730.0,2
X5K,147962.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2767365.0,30
H-LC,1507317.0,20
M-SC,1433016.0,15
H-MC,1205203.0,15
M-LC,455363.0,5
M-MC,338729.0,2
L-SC,273963.0,3
L-MC,58002.0,1
L-LC,46263.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      949425.0      7
M-SC       XY24      870221.0      7
H-SC       AR        625515.0      4
H-LC       X40       551425.0      6
H-SC       XR        529541.0      6
H-MC       XY24      473963.0      4
M-MC       XY24      338729.0      2
H-LC       X40N      325319.0      6
           XY25      254290.0      3
           AR        237013.0      2
H-MC       XY25      213695.0      2
H-SC       X40N      212449.0      4
H-MC       X40       203364.0      3
H-SC       SR        197730.0      2
           OX40N     195963.0      6
M-LC       XY24      192972.0      2
L-SC       XR        183589.0      2
M-SC       AR        161121.0      2
M-LC       X5K       134636.0      1
M-SC       XR        125592.0      2
           XY25      124305.0      1
H-MC       XR        122065.0      1
M-LC       XR        112196.0      1
H-MC       X40N       91070.0      2
L-SC       OX40N      90374.0      1
H-LC       X200       79859.0      1
M-SC       OX40N      75925.0      2
           X40        75852.0      1
L-MC       XR         58002.0      1
H-SC       MH         56742.0      1
H-MC       AR         47219.0      1
L-LC       XY25       46263.0      1
H-LC       XY24       46085.0      1
H-MC       MH         31814.0      1
           OX40N      22013.0      1
M-LC       XY25       15559.0      1
H-LC       X5K        13326.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 44.0 seconds
